In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import random

In [4]:
with open('results/no_overlap.pkl', 'rb') as file:
    results = pickle.load(file)

print("Results keys:", results.keys())
single_shot_results = results['single_shot']
ar_results = results['auto_regressive']

Results keys: dict_keys(['single_shot', 'auto_regressive'])
